# BIDS-Microscopy: empty ometiff generator to create an example dataset

The present notebook can generate an empty OME-TIFF image (only with metadata) from a sample png. The png has no metadata, so the OME-XML is created with a typed string. From this metadata, a BIDS compatible JSON file is generated. The JSON fields are filled from the OME-TIFF's metadata as a dictionary.

In [1]:
import tifffile
from ctypes import *
cdll.LoadLibrary("C:/Users/eti_m/Documents/UK-Biobank/vips-dev-8.12/bin/libgobject-2.0-0.dll")  # Change for the path where vips-dev-8.11 is
cdll.LoadLibrary("C:/Users/eti_m/Documents/UK-Biobank/vips-dev-8.12/bin/libvips-42.dll")        # idem
import pyvips
import json

## Generating the empty OME-TIFF file

In [17]:
empty_png = pyvips.Image.new_from_array([0])

empty_png = empty_png.copy()
empty_png.set_type(pyvips.GValue.gint_type, "page-height", empty_png.height)
empty_png.set_type(pyvips.GValue.gstr_type, "image-description",
f"""<?xml version="1.0" encoding="UTF-8"?><!-- Warning: this comment is an OME-XML metadata block, which contains crucial dimensional parameters and other important metadata. Please edit cautiously (if at all), and back up the original data before doing so. For more information, see the OME-TIFF web site: http://www.openmicroscopy.org/site/support/ome-model/ome-tiff/. --><OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Creator="pyvips" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd"><Experiment ID="Experiment:0"><Description>Just a description.</Description></Experiment><Instrument ID="Instrument:0"><Microscope/><Objective ID="Objective:0" Immersion="Oil" LensNA="1.4" NominalMagnification="40.0"/></Instrument><Image ID="Image:0"><AcquisitionDate>2013-01-15T17:02:40</AcquisitionDate><Pixels BigEndian="false" DimensionOrder="XYZTC" ID="Pixels:0" Interleaved="false" SignificantBits="8" SizeC="{empty_png.bands}" SizeT="{1}" SizeX="{empty_png.width}" SizeY="{empty_png.height}" SizeZ="{1}" PhysicalSizeX="{1}" PhysicalSizeXUnit="µm" PhysicalSizeY="{1}" PhysicalSizeYUnit="µm" PhysicalSizeZ="{1}" PhysicalSizeZUnit="µm" Type="uint8"></Pixels></Image></OME>""")

empty_png.tiffsave('sub-example_sample-1_TEST.ome.tif',compression="jpeg", tile=True, tile_width=512, tile_height=512, pyramid=True)

## Generating the JSON file

In [ ]:
with tifffile.TiffFile('sub-example_sample-1_SEM.ome.tif', is_ome=True) as ome:
    print(ome.ome_metadata)
    meta_dict = tifffile.xml2dict(ome.ome_metadata)['OME']
    print(meta_dict)

JSON_dict = {
        "Manufacturer" : meta_dict['Instrument']['ManufacturerAndModel'],
        "ManufacturersModelName" : meta_dict['Instrument']['ManufacturerAndModel'],
        "DeviceSerialNumber" : "n/a",
        "StationName" : "n/a",
        "SoftwareVersions" : meta_dict['Dataset']['TechnicalInfo']['SoftwareVersions'],
        "InstitutionName" : "n/a",
        "InstitutionAddress" : "n/a",
        "InstitutionalDepartmentName" : "n/a",
        "BodyPart" : meta_dict['Specimen']['Location'],
        "BodyPartDetails" : "n/a",
        "BodyPartDetailsOntology" : "n/a",
        "Environment" : meta_dict['Dataset']['Methods']['Environment'],
        "SampleEmbedding" : meta_dict['Dataset']['Methods']['SampleEmbedding'],
        "SampleFixation" : meta_dict['Dataset']['Methods']['SampleFixation'],
        "SampleStaining" : meta_dict['Dataset']['Methods']['SampleStaining'],
        "SamplePrimaryAntibody" : meta_dict['Dataset']['Methods']['SamplePrimaryAntibody'],
        "SampleSecondaryAntibody" : meta_dict['Dataset']['Methods']['SampleSecondaryAntibody'],
        "SliceThickness" : meta_dict['Dataset']['Methods']['SliceThickness'],
        "SampleExtractionProtocol" : meta_dict['Dataset']['Methods']['SampleExtractionProtocol'],
        "SampleExtractionInstitution" : meta_dict['Dataset']['Methods']['SampleExtractionInstitution'],
        "ShrinkageFactor" : meta_dict['Dataset']['Methods']['ShrinkageFactor'],
        "PixelSize" : [meta_dict['Image']['Pixels']['PhysicalSizeX'], meta_dict['Image']['Pixels']['PhysicalSizeY'], meta_dict['Image']['Pixels']['PhysicalSizeZ']],
        "PixelSizeUnits" : meta_dict['Image']['Pixels']['PhysicalSizeXUnit'],
        "ImageAcquisitionProtocol" : meta_dict['Dataset']['TechnicalInfo']['SoftwareVersions'],
        "OtherAcquisitionParameters" : "n/a"
}

with open('sub-example_sample-1_SEM.json', 'w') as fp:
    json.dump(JSON_dict, fp, indent=8)